## 1. Design patterns in action

### 1.1. Creational patterns
- creational pattern이란 **simple interface**, **safer to use** 등을 달성하기 위해 parameter initialization 등 복잡성이 높은 부분을 추상화하는 것을 의미
- creative design pattern은 object creation 과정을 제어해서 복잡성과 같은 문제들을 해결하기 위함이라고 보면 됨

#### 1.1.1. Factories
- python은 class나 function 등 모든게 객체라서 함수 인자로 넘기거나 값을 할당하는게 가능함
- 그래서 factory pattern이 사실상 필요 없다고 저자는 얘기함
- factory pattern은 아래를 참고


```python
from abc import ABCMeta, abstractmethod

class Animal(metaclass = ABCMeta):
  @abstractmethod
    def do_say(self):
    pass

class Dog(Animal):
  def do_say(self):
    print("Bhow Bhow!!")

class Cat(Animal):
  def do_say(self):
    print("Meow Meow!!")

## forest factory defined
class ForestFactory(object):
  def make_sound(self, object_type):
    return eval(object_type)().do_say()

## client code
if __name__ == '__main__':
  ff = ForestFactory()
  animal = input("Which animal should make_sound Dog or Cat?")
  ff.make_sound(animal)

'''결과'''
'''Which animal should make_sound Dog or Cat?Cat'''
'''Meow Meow!!'''
```

- 출처: https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=anciid&logNo=221793735687


#### 1.1.2. Singleton and shared state (monostate)
- 저자는 singleton pattern도 사실상 필요없거나 좋은 선택이 아니라고 하며, 뒤에서 나오겠지만 **Borg pattern**을 사용하는게 더 낫다고 함
- singleton pattern은 global variable을 만들어서 이를 공유하게 되는데 이 방식이 아래의 이유들로 인해 bad practice라고 주장
  - unittest가 어려움
  - object가 생성될때마다 상태가 바뀔수 있음 $\rightarrow$ 예측이 힘들고 여러 부작용 발생
- 그럼 singleton은 언제 필요한가?
  - module (.py)를 사용할때
  - module내에서 object를 선언해 놓으면 이를 import해서 사용하는 모든 다른 module에서 접근 가능
  - module을 얼마나 import하고 사용되는지간에 결국 `sys.modules`로 load되는건 단 하나 뿐이라서 python module은 이미 singleton이라고 한다

##### shared state
- 하나의 객체를 생성해서 singleton을 갖는 것보다 여러 instance간에 data를 복제하는게 낫다고 함
- monostate pattern은 객체간의 정보들이 완전히 투명한 방식으로 동기화되기 때문에 앞서 얘기한 singleton의 단점들에서 상대적으로 자유로움

In [ ]:
import logging

logging.basicConfig()
logger = logging.getLogger(__name__)

class GitFetcher:
    _current_tag = None

    def __init__(self, tag):
        self.current_tag = tag

    @property
    def current_tag(self):
        if self._current_tag is None:
            raise AttributeError("tag was never set")
        return self._current_tag

    @current_tag.setter
    def current_tag(self, new_tag):
        # class attribute을 update하기 때문에 instance간에 상태 공유가 가능
        self.__class__._current_tag = new_tag

    def pull(self):
        logger.info("pulling from %s", self.current_tag)
        return self.current_tag


In [ ]:
f1 = GitFetcher(0.1)
f2 = GitFetcher(0.2)

print(f1.pull(), f2.pull())

0.2 0.2


In [ ]:
f1.current_tag = 0.3

print(f1.pull(), f2.pull())

0.3 0.3


* 만약 더 많은 class attribute을 사용해야 될 일이 있다면?
* descriptor를 사용하면 코드는 더 길어질수 있지만 응집력이 더 높아지고 단일책임원칙에도 잘 부합하게 됨
  - descriptor: `__get__`, `__set__`, `__delete__`중 하나 이상의 magic method를 구현한 class였음

In [ ]:
class SharedAttribute:
    def __init__(self, initial_value=None):
        self.value = initial_value
        self._name = None

    def __get__(self, instance, owner):
        if instance is None:
            return self
        if self.value is None:
            raise AttributeError(f"{self._name} was never set")
        return self.value

    def __set__(self, instance, new_value):
        self.value = new_value

    def __set_name__(self, owner, name):
        self._name = name


class GitFetcher:

    # descriptor (current_tag, current_branch)
    current_tag = SharedAttribute()
    current_branch = SharedAttribute()

    def __init__(self, tag, branch=None):
        self.current_tag = tag
        self.current_branch = branch

    def pull(self):
        logger.info("pulling from %s", self.current_tag)
        return self.current_tag


In [ ]:
f1 = GitFetcher(0.1, branch='dev')
print(f1.current_tag, f1.current_branch)

f2 = GitFetcher(0.2, branch='main')
print(f2.current_tag, f2.current_branch)

print(f1.pull(), f2.pull())
print(f1.current_branch, f2.current_branch)

0.1 dev
0.2 main
0.2 0.2
main main


##### The borg pattern
- 앞서 예제도 충분하지만 정말로 singleton이 필요할때의 대안은 borg pattern임
- idea: 같은 class에서 만들어진 instance들 사이의 attribute을 전부 공유 가능하도록 객체를 만든다
  - 아래 코드를 보면 dictionary를 공유하도록 만들었음
  - 이 로직을 base class에 넣으면 안되나? $\rightarrow$ Tag랑 Branch간에 값들이 mix되기 때문에 다른 방식을 사용해야됨 (바로 다음 Mixin 참고)

In [23]:
class BaseFetcher:
    def __init__(self, source):
        self.source = source


class TagFetcher(BaseFetcher):
    _attributes = {}

    def __init__(self, source):        
        self.__dict__ = self.__class__._attributes
        super().__init__(source)

    def pull(self):
        logger.info("pulling from tag %s", self.source)
        return f"Tag = {self.source}"


class BranchFetcher(BaseFetcher):
    _attributes = {}

    def __init__(self, source):
        self.__dict__ = self.__class__._attributes
        super().__init__(source)

    def pull(self):
        logger.info("pulling from branch %s", self.source)
        return f"Branch = {self.source}"


In [25]:
f1 = TagFetcher(0.1)
f2 = TagFetcher(0.2)

b1 = BranchFetcher('dev')
b2 = BranchFetcher('main')

In [26]:
TagFetcher._attributes

{'source': 0.2}

In [27]:
BranchFetcher._attributes

{'source': 'main'}

In [41]:
class SharedAllMixin:
    def __init__(self, *args, **kwargs):
        try:
            self.__class__._attributes
        except AttributeError:
            self.__class__._attributes = {}

        # self.__class__ => TagFetcher 또는 BranchFetcher
        self.__dict__ = self.__class__._attributes
        # super class => BaseFetcher
        super().__init__(*args, **kwargs)


class BaseFetcher:
    def __init__(self, source):
        self.source = source


class TagFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):        
        logger.info("pulling from tag %s", self.source)
        return f"Tag = {self.source}"


class BranchFetcher(SharedAllMixin, BaseFetcher):
    def pull(self):
        logger.info("pulling from branch %s", self.source)
        return f"Branch = {self.source}"


In [42]:
f1 = TagFetcher(0.1)
b1 = BranchFetcher('dev')

In [43]:
TagFetcher._attributes

{'source': 0.1}

In [44]:
BranchFetcher._attributes

{'source': 'dev'}

#### 1.1.3. Builder
- 객체 생성에 필요한 모든 복잡한 초기화 절차들을 추상화하는 패턴
- 다른 여러 객체들과 함께 상호작용해야 하는 복잡한 객체를 생성할때 주로 사용되는 패턴임
- 유저가 필요한 여러 보조 객체들을 생성해서 필요한 모든 절차를 수행하는 대신 single step으로 이를 다 처리해주는 것이 목적임


```python
from enum import Enum
import time

PizzaProgress = Enum('PizzaProgress', 'queued preparation baking ready')
PizzaDough = Enum('PizzaDough', 'thin thick')
PizzaSauce = Enum('PizzaSauce', 'tomato creme_fraiche')
PizzaTopping = Enum('PizzaTopping', 
                    'mozzarella double_mozzarella bacon ham mushrooms red_onion oregano')
STEP_DELAY = 3 # in seconds for the sake of the example


class Pizza:
    def __init__(self, name):
        self.name = name
        self.dough = None
        self.sauce = None
        self.topping = []

    def __str__(self):
        return self.name

    def prepare_dough(self, dough):
        self.dough = dough
        print(f'preparing the {self.dough.name} dough of your {self}...')
        time.sleep(STEP_DELAY)
        print(f'done with the {self.dough.name} dough')

        
class MargaritaBuilder:
    def __init__(self):
        self.pizza = Pizza('margarita')
        self.progress = PizzaProgress.queued
        self.baking_time = 5 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thin)

    def add_sauce(self):
        print('adding the tomato sauce to your margarita...')
        self.pizza.sauce = PizzaSauce.tomato
        time.sleep(STEP_DELAY)
        print('done with the tomato sauce')

    def add_topping(self):
        topping_desc = 'double mozzarella, oregano'
        topping_items = (PizzaTopping.double_mozzarella, PizzaTopping.oregano)
        print(f'adding the topping ({topping_desc}) to your margarita')
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print(f'done with the topping ({topping_desc})')

    def bake(self):
        self.progress = PizzaProgress.baking
        print(f'baking your margarita for {self.baking_time} seconds')
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your margarita is ready')

        
class CreamyBaconBuilder:
    def __init__(self):
        self.pizza = Pizza('creamy bacon')
        self.progress = PizzaProgress.queued
        self.baking_time = 7 # in seconds for the sake of the example

    def prepare_dough(self):
        self.progress = PizzaProgress.preparation
        self.pizza.prepare_dough(PizzaDough.thick)

    def add_sauce(self):
        print('adding the crème fraîche sauce to your creamy bacon')
        self.pizza.sauce = PizzaSauce.creme_fraiche
        time.sleep(STEP_DELAY)
        print('done with the crème fraîche sauce')

    def add_topping(self):
        topping_desc = 'mozzarella, bacon, ham, mushrooms, red onion, oregano'
        topping_items =  (PizzaTopping.mozzarella,
                          PizzaTopping.bacon,
                          PizzaTopping.ham,
                          PizzaTopping.mushrooms,
                          PizzaTopping.red_onion, 
                          PizzaTopping.oregano)
        print(f'adding the topping ({topping_desc}) to your creamy bacon')
        self.pizza.topping.append([t for t in topping_items])
        time.sleep(STEP_DELAY)
        print(f'done with the topping ({topping_desc})')

    def bake(self):
        self.progress = PizzaProgress.baking
        print(f'baking your creamy bacon for {self.baking_time} seconds')
        time.sleep(self.baking_time)
        self.progress = PizzaProgress.ready
        print('your creamy bacon is ready')

        
class Waiter:
    def __init__(self):
        self.builder = None

    def construct_pizza(self, builder):
        self.builder = builder
        steps = (builder.prepare_dough, 
                 builder.add_sauce, 
                 builder.add_topping, 
                 builder.bake)
        [step() for step in steps]

    @property
    def pizza(self):
        return self.builder.pizza

        
def validate_style(builders):
    try:
        input_msg = 'What pizza would you like, [m]argarita or [c]reamy bacon? '
        pizza_style = input(input_msg)
        builder = builders[pizza_style]()
        valid_input = True
    except KeyError:
        error_msg = 'Sorry, only margarita (key m) and creamy bacon (key c) are available'
        print(error_msg)
        return (False, None)
    return (True, builder)

    
def main():
    builders = dict(m=MargaritaBuilder, c=CreamyBaconBuilder)
    valid_input = False
    while not valid_input:
        valid_input, builder = validate_style(builders)
    print()
    waiter = Waiter()
    waiter.construct_pizza(builder)
    pizza = waiter.pizza
    print()
    print(f'Enjoy your {pizza}!')

```

- 출처: https://bit.ly/3FUFyc2

### 1.2. Structural patterns
- 인터페이스에 복잡성을 추가하지 않으면서도, 기능을 추가하며 간단한 인터페이스를 유지하고 싶을때 유용하게 사용되는 패턴
- 여러개의 객체들을 compose하거나, 단순하고 응집력있는 인터페이스를 gathering

#### 1.2.1. Adaptor
- wrapper로도 알려져 있으며, 2개 이상의 incompatible한 인터페이스를 결합해서 문제를 해결하는 패턴임
- 예)
  - `fetch`라는 method를 갖는 data source class A가 있고, 새로운 데이터에 대한 data source class B를 구현하는 상황
- 해결 방법
  1. B가 A를 상속받아서 `fetch`를 사용
  2. B에 A를 composition해서 `fetch`를 사용
- 상속은 coupling이 생기게 되고, 종종 명확하지 않은 경우가 있기 때문에 composition이 더 선호되기도 함

In [55]:
import logging

logging.basicConfig()
logger = logging.getLogger()


class UsernameLookup:
    # 필요한 method
    def search(self, user_namespace):
        logger.info("looking for %s", user_namespace)


# 상속을 통한 문제 해결
class UserSourceV1(UsernameLookup):
    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"


# composition을 통한 문제 해결
class UserSourceV2:
    def __init__(self, username_lookup: UsernameLookup) -> None:
        self.username_lookup = username_lookup

    def fetch(self, user_id, username):
        user_namespace = self._adapt_arguments(user_id, username)
        return self.username_lookup.search(user_namespace)

    @staticmethod
    def _adapt_arguments(user_id, username):
        return f"{user_id}:{username}"

In [56]:
v1 = UserSourceV1()
v2 = UserSourceV2(UsernameLookup())

In [57]:
v1.fetch(user_id=1, username='user1')

In [58]:
v2.fetch(user_id=1, username='user1')

#### 1.2.2. Composite
- 잘 정의된 base object가 있고, 다른 container object가 base object들을 그룹화 함
- tree와 같은 계층 구조
  - basic object: leave node
  - composed object: intermediate node

In [59]:
from typing import Iterable, Union


class Product:
    def __init__(self, name, price):
        self._name = name
        self._price = price

    @property
    def price(self):
        return self._price


class ProductBundle:
    def __init__(
        self,
        name,
        perc_discount,
        *products: Iterable[Union[Product, "ProductBundle"]]
    ) -> None:
        self._name = name
        self._perc_discount = perc_discount
        self._products = products

    @property
    def price(self):
        total = sum(p.price for p in self._products)
        return total * (1 - self._perc_discount)


In [60]:
products = [
    Product('apple', 1000),
    Product('banana', 2000),
    Product('melon', 3000),
]

In [73]:
bundle = ProductBundle('cart', 0.2, *products)

In [74]:
bundle.price

4800.0

#### 1.2.3. Decorator
- 상속 없이 특정 객체의 기능을 확장해서 사용하고 싶을때 사용되는 패턴

In [113]:
class DictQuery:
    def __init__(self, **kwargs):
        self._raw_query = kwargs

    def render(self) -> dict:
        return self._raw_query


class QueryEnhancer:
    def __init__(self, query: DictQuery):
        self.decorated = query

    def render(self):
        return self.decorated.render()


class RemoveEmpty(QueryEnhancer):
    def render(self):
        original = super().render()
        return {k: v for k, v in original.items() if v}


class CaseInsensitive(QueryEnhancer):
    def render(self):        
        original = super().render()
        return {k: v.lower() for k, v in original.items()}


In [114]:
 original = DictQuery(
     key="value", empty="", none=None,
     upper="UPPERCASE", title="Title"
)

In [115]:
new_query = CaseInsensitive(RemoveEmpty(original))

In [116]:
original.render()

{'empty': '',
 'key': 'value',
 'none': None,
 'title': 'Title',
 'upper': 'UPPERCASE'}

In [117]:
# QueryEnhancer(RemoveEmpty(original)).render()
#   - self.decorated = RemoveEmpty(original)
#   - self.decorated.render()
#   - RemoveEmpty(original).render()
# QueryEnhancer(original).render()
#   - self.decorated = original
#   - self.decorated.render()
#   - DictQuery(original).render()

new_query.render()

{'key': 'value', 'title': 'title', 'upper': 'uppercase'}

* 아래와 같이 function을 사용해서 decorator pattern을 적용하는 것도 가능
* 그러나 object 내의 data를 참고해야 하거나 각 class 간의 계층적 구조를 명시적으로 나타내고 싶은 상황에서는 위와 같은 object-oriented approach를 사용하는게 좋음

In [82]:
from typing import Callable, Dict, Iterable


class DictQuery:
    def __init__(self, **kwargs):
        self._raw_query = kwargs

    def render(self) -> dict:
        return self._raw_query


class QueryEnhancer:
    def __init__(
        self,
        query: DictQuery,
        *decorators: Iterable[Callable[[Dict[str, str]], Dict[str, str]]]
    ) -> None:
        self._decorated = query
        self._decorators = decorators

    def render(self):
        current_result = self._decorated.render()
        for deco in self._decorators:
            current_result = deco(current_result)
        return current_result


# decorator 1
def remove_empty(original: dict) -> dict:
    return {k: v for k, v in original.items() if v}


# decorator 2
def case_insensitive(original: dict) -> dict:
    return {k: v.lower() for k, v in original.items()}


In [83]:
query = DictQuery(
    foo="bar", empty="", none=None,
    upper="UPPERCASE", title="Title"
)

In [84]:
QueryEnhancer(query, remove_empty, case_insensitive).render()

{'foo': 'bar', 'title': 'title', 'upper': 'uppercase'}

#### 1.2.4. Facade
- object들 사이에 상호작용이 복잡할떄 이를 단순화 하기 위해 사용되는 패턴
- N개의 object들이 있을때 N개의 각 object들에게 명령을 하는 대신 facade와 상호작용함으로써 명령어들을 실행하게 됨
- 실생활 예
  - 컴퓨터를 켤 경우 복잡한 내부 동작은 다 숨겨지고 전원버튼만 누르는 것으로 사용자에게 간단한 인터페이스를 제공

```python
class Subsystem1:
    def play(self):
        pass

class Subsystem2:
    def stop(self):
        pass

class Subsystem3:
    def pause(self):
        pass



class Facade:
    def __init__(self):
        self.one = Subsystem1()
        self.two = Subsystem2()
        self.three = Subsystem3()

    def exec(self):
        self.one.play()
        self.three.pause()
        self.two.stop()
```

- 출처: https://lee-seul.github.io/concept/design-patterns/2017/03/17/design-pattern-04-facade.html

### 1.3. Behavioral patterns
- 객체들이 어떻게 협력하고, 통신하고 런타임 인터페이스가 어떻게 되는지를 정의하는 패턴
- 상속을 통한 정적으로 또는 composition을 통한 동적으로 수행 가능

#### 1.3.1. Chain of responsibility
- 앞의 객체가 연산을 수행하지 못할때 같은 인터페이스의 다른 객체에게 연산을 수행하도록 전달하는 패턴

In [1]:
import re


class Event:
    pattern = None

    def __init__(self, next_event=None):
        self.successor = next_event

    def process(self, logline: str):
        # 현재 이벤트 처리
        if self.can_process(logline):
            return self._process(logline)

        # 다음 이벤트가 있으면 다음 이벤트 처리
        if self.successor is not None:            
            return self.successor.process(logline)

    def _process(self, logline: str) -> dict:
        parsed_data = self._parse_data(logline)
        return {
            "type": self.__class__.__name__,
            "id": parsed_data["id"],
            "value": parsed_data["value"],
        }

    @classmethod
    def can_process(cls, logline: str) -> bool:
        return cls.pattern.match(logline) is not None

    @classmethod
    def _parse_data(cls, logline: str) -> dict:
        return cls.pattern.match(logline).groupdict()


class LoginEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+login\s+(?P<value>\S+)")


class LogoutEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+logout\s+(?P<value>\S+)")


class SessionEvent(Event):
    pattern = re.compile(r"(?P<id>\d+):\s+log(in|out)\s+(?P<value>\S+)")


In [2]:
chain = LogoutEvent(LoginEvent())

In [3]:
chain.process('567: login User')

{'id': '567', 'type': 'LoginEvent', 'value': 'User'}

In [4]:
# Session Event가 우선순위가 더 높다면 Chain의 순서를 이런식으로 변경
chain = SessionEvent(LoginEvent(LogoutEvent()))

In [5]:
chain.process('567: login User')

{'id': '567', 'type': 'SessionEvent', 'value': 'User'}

#### 1.3.2. The template method
- 앞서 chain of responsibility와 거의 동일
- code reusability를 높이거나 다형성을 보존하며 유연한 구조를 만드는데 자주 사용되는 패턴
- child class에서 변경이 필요할시 overwrite해서 구현하는 방식

```python
from abc import abstractmethod, ABC


class AbstractParser(ABC):
    url = None

    def get_url(self):
        if self.url is None:
            raise ValueError
        return self.url

    def get_content(self):
        _url = self.get_url()
        print('Send a request to _url to import content.')

    @abstractmethod
    def parsing(self, contents):
        pass

    def hook1(self, data):
        pass

    def execute(self):
        contents = self.get_content()
        self.hook1(self.parsing(contents))


class ConingguParser(AbstractParser):
    url = 'https://www.coningg.com/rss'

    def parsing(self, contents):
        print('parsing!')


class DolgonetParser(AbstractParser):
    url = 'http://dolgo.net/rss'

    def parsing(self, contents):
        print('parsing!')

    def hook1(self, data):
        pass
```

- 출처: https://www.coninggu.com/11

#### 1.3.3. Command
- `__call__()`, `do()`, `execute()` 등 실행에 필요한 command를 method로 갖는 command 객체들을 갖는 객체를 만들어 놓고 필요에 따라 처리하는 패턴

```python
class Light:
    def __init__(self, location: str):
        self.location = location

    def on(self):
        print(f"{self.location} 전등 켜짐")

    def off(self):
        print(f"{self.location} 전등 꺼짐")


class MusicPlayer:
    def __init__(self, location: str):
        self.location = location

    def on(self):
        print(f"{self.location} 음악 플레이어 켜짐")

    def off(self):
        print(f"{self.location} 음악 플레이어 꺼짐")


class Command(metaclass=ABCMeta):
    @abstractmethod
    def execute(self):
        pass


class LightOnCommand(Command):
    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        self.light.on()

    def undo(self):
        self.light.off()


class LightOffCommand(Command):
    def __init__(self, light: Light):
        self.light = light

    def execute(self):
        self.light.off()

    def undo(self):
        self.light.on()


class MusicPlayerOnCommand(Command):
    def __init__(self, music_player: MusicPlayer):
        self.music_player = music_player

    def execute(self):
        self.music_player.on()

    def undo(self):
        self.music_player.off()


class MusicPlayerOffCommand(Command):
    def __init__(self, music_player: MusicPlayer):
        self.music_player = music_player

    def execute(self):
        self.music_player.off()

    def undo(self):
        self.music_player.on()


class RemoteControl:
    def __init__(self):
        self.buttons = [None] * 10
        self.last_command = None

    def setCommand(self, index: int, command: Command):
        self.buttons[index] = command

    def pressButton(self, index):
        self.buttons[index].execute()
        self.last_command = self.buttons[index]

    def pressUndoButton(self):
        self.last_command.undo()


# 책상 위에 전등과 음악 플레이어가 있다.
light = Light("책상 위")
music_player = MusicPlayer("책상 위")

# 리모콘에 명령어 세팅을 한다.
remote_control = RemoteControl()
remote_control.setCommand(0, LightOnCommand(light))
remote_control.setCommand(1, LightOffCommand(light))
remote_control.setCommand(2, MusicPlayerOnCommand(music_player))
remote_control.setCommand(3, MusicPlayerOffCommand(music_player))

# 사용자가 각 버튼을 순서대로 누른다.
remote_control.pressButton(0) # 책상 위 전등 켜짐
remote_control.pressButton(2) # 책상 위 음악 플레이어 켜짐
remote_control.pressButton(1) # 책상 위 전등 꺼짐
remote_control.pressButton(3) # 책상 위 음악 플레이어 꺼짐

# 마지막 버튼 실행 취소를 한다.
remote_control.pressUndoButton() # 책상 위 음악 플레이어 켜짐
```

- 출처: https://dailyheumsi.tistory.com/217

#### 1.3.4. State
- 상태를 관리하는데 도움을 주는 디자인 패턴
- 예: merge request 규칙
  - open에서 closed로 상태가 바뀌면 approval 목록이 모두 삭제
  - MR이 막 open된 상태면 approval 수는 0이어야 됨 (reopen이거나 완전 처음 open인거랑 상관 없이)
  - merge가 완료되면 source branch를 제거하고, invalid transition을 불가능하게 설정

- 만약 `MergeRequest` 클래스에 위 규칙을 다 구현한다면, 수많은 `if` 문들이 포함될것이며, 코드를 이해하기 어려워 질 것임 (해당 클래스가 너무 많은 책임을 지게 됨)

- 따라서 이를 더 작은 객체들로 나누어서, 책임을 분산하는게 더 좋은 구조이며 State object를 통해 이를 만족시킬 수 있음

In [158]:
import abc
import logging

logging.basicConfig()
logger = logging.getLogger()
logger.setLevel(logging.INFO)


class InvalidTransitionError(Exception):
    """Raised when trying to move to a target state from an unreachable source
    state.
    """


class MergeRequestState(abc.ABC):
    def __init__(self, merge_request):
        self._merge_request = merge_request

    @abc.abstractmethod
    def open(self):
        ...

    @abc.abstractmethod
    def close(self):
        ...

    @abc.abstractmethod
    def merge(self):
        ...

    def __str__(self):
        return self.__class__.__name__


class Open(MergeRequestState):
    def open(self):
        self._merge_request.approvals = 0

    def close(self):
        self._merge_request.approvals = 0
        self._merge_request.state = Closed

    def merge(self):
        logger.info("merging %s", self._merge_request)
        logger.info("deleting branch %s", self._merge_request.source_branch)
        self._merge_request.state = Merged


class Closed(MergeRequestState):
    def open(self):
        logger.info("reopening closed merge request %s", self._merge_request)
        self._merge_request.state = Open

    def close(self):
        """Current state."""

    def merge(self):
        raise InvalidTransitionError("can't merge a closed request")


class Merged(MergeRequestState):
    def open(self):
        raise InvalidTransitionError("already merged request")

    def close(self):
        raise InvalidTransitionError("already merged request")

    def merge(self):
        """Current state."""


class MergeRequest:
    def __init__(self, source_branch: str, target_branch: str) -> None:
        self.source_branch = source_branch
        self.target_branch = target_branch
        self._state = None
        self.approvals = 0
        self.state = Open

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, new_state_cls):
        self._state = new_state_cls(self)

    def open(self):
        return self.state.open()

    def close(self):
        return self.state.close()

    def merge(self):
        return self.state.merge()

    def __str__(self):
        return f"{self.target_branch}:{self.source_branch}"


In [159]:
mr = MergeRequest('dev', 'main')
print(mr)

main:dev


In [160]:
mr.open()
print(mr.approvals)

0


In [161]:
mr.approvals = 3
mr.close()
print(mr.approvals)

0


In [162]:
mr.open()
print(mr.approvals)

INFO:root:reopening closed merge request main:dev


0


In [163]:
mr.merge()

INFO:root:merging main:dev
INFO:root:deleting branch dev


In [164]:
mr.close()

InvalidTransitionError: ignored

In [176]:

class MergeRequestState(abc.ABC):
    def __init__(self, merge_request):
        self._merge_request = merge_request

    @abc.abstractmethod
    def open(self):
        ...

    @abc.abstractmethod
    def close(self):
        ...

    @abc.abstractmethod
    def merge(self):
        ...

    def __str__(self):
        return self.__class__.__name__


class Open(MergeRequestState):
    def open(self):
        self._merge_request.approvals = 0

    def close(self):
        self._merge_request.approvals = 0
        self._merge_request.state = Closed

    def merge(self):
        logger.info("merging %s", self._merge_request)
        logger.info("deleting branch %s", self._merge_request.source_branch)
        self._merge_request.state = Merged


class Closed(MergeRequestState):
    def open(self):
        logger.info("reopening closed merge request %s", self._merge_request)
        self._merge_request.state = Open

    def close(self):
        """Current state."""

    def merge(self):
        raise InvalidTransitionError("can't merge a closed request")


class Merged(MergeRequestState):
    def open(self):
        raise InvalidTransitionError("already merged request")

    def close(self):
        raise InvalidTransitionError("already merged request")

    def merge(self):
        """Current state."""


class MergeRequest:
    def __init__(self, source_branch: str, target_branch: str) -> None:
        self.source_branch = source_branch
        self.target_branch = target_branch
        self._state: MergeRequestState
        self.approvals = 0
        self.state = Open

    @property
    def state(self):
        return self._state

    @state.setter
    def state(self, new_state_cls):
        self._state = new_state_cls(self)

    @property
    def status(self):
        return str(self.state)

    # open/close/merge와 같은 불필요한 코드 제거
    def __getattr__(self, method):
        return getattr(self.state, method)

    def __str__(self):
        return f"{self.target_branch}:{self.source_branch}"


In [177]:
mr = MergeRequest('dev', 'main')
print(mr)

main:dev


In [178]:
mr.status

'Open'

In [179]:
mr.approvals = 3
print(mr.approvals)

3


In [180]:
mr.close()
print(mr.approvals)

0


In [181]:
mr.open()
print(mr.approvals)

INFO:root:reopening closed merge request main:dev


0


In [182]:
mr.merge()

INFO:root:merging main:dev
INFO:root:deleting branch dev


In [183]:
mr.close()

InvalidTransitionError: ignored

## 2. The null object pattern
- function이나 method는 동일 타입의 객체를 return해야 됨
- 이게 보장되면 client는 추가 검사없이 다형성이 보장된 객체를 사용할수 있음
- 앞서 봤던 Chain of Responsibility에서 output은 dictionary임
  - 사용자는 dictionary를 기대하고 `result.keys()`를 호출하는데 만약 log line을 처리하지 못해서 None이 반환되면 `AttributeError: 'NoneType' object has no attribute 'keys'` 에러가 발생하게 됨 
  - 따라서 log line을 처리하지 못하면 `{}`를 반환해야됨
- 이렇게 해야 runtime에러를 피할수 있고, test를 하기도 수월하며, 왜 해당 상태가 return됐는지 디버깅도 수월함

In [185]:
event = SessionEvent(LoginEvent(LogoutEvent()))

In [190]:
result = event.process('1234: login val1')
print(result)

{'type': 'SessionEvent', 'id': '1234', 'value': 'val1'}


In [192]:
result = event.process('1234: log ??')
print(result)

None


## 3. Final thoughts about design patterns
- 디자인 패턴이 항상 좋을까?
- 아래와 같은 반대 의견들이 있음
  - 파이썬에서 쉽게 수행 가능한 작업들이 불가능한 언어들을 위해 디자인 패턴이 생겨났음
  - 디자인 패턴은 design solution을 강제해서 더 나은 방식이 나올 수 있는 것을 제한하고 있음

### 3.1. The influence of patterns over the design
- design pattern이 좋고 나쁨은 그걸 어떻게 사용하냐에 따라 달렸음
- 적절하지 않게 사용하거나 over-engineering 하는게 문제이지 design pattern 자체가 나쁜건 아님
- 좋은 소프트웨어를 만드는건 미래의 요구사항이 아니라 단지 현재 직면한 문제를 풀어내는 것이며,
미래의 수정사항들에 대해 충분히 유연하면 됨  
- 도메인에 맞게 디자인을 해서 문제하고 거기서부터 어떤 디자인 패턴이 존재하는지 확인하는 순서가 좋음

### 3.2. Names in our models
- 디자인이 좋고 코드가 명료하다면 그 자체로 충분하며, 디자인패턴 이름을 굳이 드러낼 필요가 없음
  - 코드가 의도한대로 잘 동작한다면 유저나 다른 개발자들이은 거기에 숨어져 있는 디자인 패턴까지 알 필요는 없음
  - 의도적으로 디자인패턴 이름을 내비치는건 intention revealing princple을 위반하는것
  - 예를 들어, class가 query를 의미한다면 `Query`, `EnhancedQuery`로 충분하지 `EnhancedQueryDecorator`와 같은 이름을 사용할 필요가 없음 (오히려 혼란만 커질수도)
- docstring에 관련 내용을 적어서 의도가 어떤가에 얘기하는 정도는 괜찮음
- 최고의 디자인패턴은 우리가 생각할 필요조차 없게 이미 잘 추상화 되어 있는것 (iterator design pattern)